In [ ]:
import pandas as pd

In [ ]:
valores = pd.read_csv('/content/anond6aa91eca5dc7eab2a41d9b43cdd5c6f5c34a42b5bdfbe79ef2a26cd413430d5.csv')
personas = pd.read_csv('/content/ventas 8 dic - SALUD.csv')

In [ ]:
print(personas.columns.tolist())

['codigo_producto', 'clave_agente', 'numero_poliza', 'fecha_emision', 'nombre_producto', 'nombre_opcion_poliza', 'prima', 'tipo_documento', 'NUMERO_DOCUMENTO', 'NOMBRE', 'CORREO_1', 'CELULAR_1', 'FECHA_PROCESO']


In [ ]:
personas['CELULAR_1'] = personas['CELULAR_1'].astype(str).str.replace('.0', '', regex=False)

In [ ]:
agrupado = personas.groupby('numero_poliza').agg({
    'clave_agente': 'first',
    'codigo_producto': 'first',
    'fecha_emision': 'first',
    'nombre_producto': 'first',
    'nombre_opcion_poliza': 'first',
    'tipo_documento': 'first',
    'NUMERO_DOCUMENTO': lambda x: ', '.join(sorted(set(x.dropna().astype(str)))),
    'NOMBRE': lambda x: ', '.join(sorted(set(x.dropna().astype(str)))),
    'CORREO_1': lambda x: ', '.join(sorted(set(x.dropna().astype(str)))),
    'CELULAR_1': lambda x: next(iter(sorted(set(x.dropna().astype(str)))) , ''),
    'FECHA_PROCESO': 'first'
}).reset_index()

In [ ]:
resultado = pd.merge(agrupado, valores[['numero_poliza', 'PRIMA']], on='numero_poliza', how='left')
resultado = resultado.rename(columns={
    'PRIMA': 'Prima_totalizada',
    'FECHA_PROCESO': 'FECG'
})

In [ ]:
def expandir_columna(df, columna_base, prefijo):
    df[columna_base] = df[columna_base].fillna('')
    listas = df[columna_base].apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
    max_items = listas.apply(len).max()
    nuevas_cols = pd.DataFrame(listas.tolist(), columns=[f"{prefijo}_{i+1}" for i in range(max_items)])
    return pd.concat([df.drop(columns=[columna_base]), nuevas_cols], axis=1)

resultado = expandir_columna(resultado, 'NUMERO_DOCUMENTO', 'Numero_documento')
resultado['telefono'] = resultado['CELULAR_1']

In [ ]:
columnas_finales = [
    'emisiones',
    'revision agente',
    'codigo_producto',
    'clave_agente',
    'numero_poliza',
    'fecha_emision',
    'nombre_producto',
    'nombre_opcion_poliza',
    'Prima_totalizada',
    'CORREO_1',
    'Numero_documento_1',
    'tipo_documento',
    'Numero_documento_2',
    'FECG',
    'telefono'
]

for col in columnas_finales:
    if col not in resultado.columns:
        resultado[col] = ''

resultado_final = resultado[columnas_finales]

In [ ]:

resultado_final.to_csv('Salud cruzado.csv', index=False, encoding='utf-8-sig')
print("Archivo generado: Salud cruzado.csv")

Archivo generado: Salud cruzado.csv
